In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [65]:
import numpy as np
import random
import re
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import pandas as pd
import nltk
from nltk.tokenize import WordPunctTokenizer
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [66]:
pos_file_path = "/content/drive/MyDrive/Colab Notebooks/twitter-datasets/train_pos.txt"
neg_file_path = "/content/drive/MyDrive/Colab Notebooks/twitter-datasets/train_neg.txt"
test_file_path = "/content/drive/MyDrive/Colab Notebooks/twitter-datasets/test_data.txt"
pos_file_path_prep = "/content/drive/MyDrive/Colab Notebooks/twitter-datasets/train_pos_prep.txt"
neg_file_path_prep = "/content/drive/MyDrive/Colab Notebooks/twitter-datasets/train_neg_prep.txt"
test_file_path_prep = "/content/drive/MyDrive/Colab Notebooks/twitter-datasets/test_data_prep.txt"
pos_file_path_prep_deep = "/content/drive/MyDrive/Colab Notebooks/twitter-datasets/train_pos_prep_deep.txt"
neg_file_path_prep_deep = "/content/drive/MyDrive/Colab Notebooks/twitter-datasets/train_neg_prep_deep.txt"
test_file_path_prep_deep = "/content/drive/MyDrive/Colab Notebooks/twitter-datasets/test_data_prep_deep.txt"

In [67]:
def rewrite_abbreviations(line):
    line = line.replace('can\'t', 'cannot')
    line = line.replace('ain\'t', 'is not')
    line = line.replace('n\'t', ' not')  
    line = line.replace('i\'m', 'i am') 
    line = line.replace('\'re', ' are') 
    line = line.replace('it\'s', 'it is') 
    line = line.replace('that\'s', 'that is') 
    line = line.replace('\'ll', ' will') 
    line = line.replace('\'ve', ' have')
    line = line.replace('\'d', ' would')
    line = line.replace('he\'s', 'he is') 
    line = line.replace('she\'s', 'she is')     
    line = line.replace('what\'s', 'what is')  
    line = line.replace('who\'s', 'who is') 
    line = line.replace('thanx', 'thanks')  
    line = line.replace('&', 'and')
    line = line.replace('wont', 'will not')
    line = line.replace('wasnt', 'was not')
    line = line.replace('isnt', 'is not')
    line = line.replace('werent', 'were not')
    line = line.replace('didnt', 'did not')
    line = line.replace('couldnt', 'could not')
    line = line.replace('shouldnt', 'should not')
    line = line.replace('wouldnt', 'would not')
    line = line.replace('havent', 'have not')
    line = line.replace('youre', 'you are')
    line = line.replace('youve', 'you have')
    line = line.replace('theyre', 'they are')
    line = line.replace('theyve', 'they have')
    line = line.replace('dunno', 'do not know')
    line = line.replace('youll', 'you will')
    line = line.replace('theyll', 'they will')
    return line

def remove_hashtags(line):
    line = re.sub("@[A-Za-z0-9_]+", '', line)
    line = re.sub("#[A-Za-z0-9_]+", '', line)
    return line

def replace_numbers(line):
    line = re.sub('[0-9]{5,}', 'numhuge', line)
    line = re.sub('[0-9]{4}', 'numlarge', line)
    line = re.sub('[0-9]{3}', 'nummedium', line)
    line = re.sub('[0-9]{2}', 'numsmall', line)
    return line

def remove_stop_word(line):
    stop_words = set(['<user>', '<url>'])
    for word in stop_words:
      line = line.replace(word, '')
    return line
    
def remove_punctuations(line):
    punctuations = set([
        '\"', '$', '%', '&', '\\', '\'', '(', ')', '*', '+',
        ',', '-', '.', '/', '^', '_', '`', '{', '|', ':', 
        ';', '<', '=', '>', '@', '[', ']', '}', '~', '#'])
 #   res=" "
 #   for ele in line:
 #     if ele not in punctuations:
 #       res+=ele
 #   return res
  
    for word in punctuations:
      line = line.replace(word, '')
    return line

def replace_emoticons(line):
    emoticons_good = set([
        ':-)', ':)', ';)', ':o)', ':]', ':3', '<3', ':c)', ':>', '=]', '8)', '=)', ':}',
        ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
        '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
        'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
        '<3', 'xo', 'lol', 'hahaha', 'haha'])
    emoticons_bad = set([
        ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
        ':-[', ':-<', '=\\', '=/', ':/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
        ':c', ':{', '>:\\', ';(']) 
    for word in emoticons_good:
        line = line.replace(word, 'good')
    for word in emoticons_bad:    
        line = line.replace(word, 'bad')
    return line

def remove_spaces(line):
    line = re.sub(' +', ' ', line)
    line = line.lstrip()
    return line

def stemming_words(line):
    porter_stemmer = PorterStemmer()
    spl = line.split()
    new = []
    for word in spl:
        new.append(porter_stemmer.stem(word))
    line = " ".join(new)
    return line

In [68]:
# preprocessing for deep learning models: no word stemming

def preprocessing_deep(line):
    line = replace_emoticons(line)
    line = rewrite_abbreviations(line)
    line = remove_hashtags(line)
    line = replace_numbers(line)        
    line = remove_stop_word(line)
    line = remove_punctuations(line)
    line = remove_spaces(line)
    return line

In [69]:
with open(pos_file_path, encoding='utf8') as f, open(pos_file_path_prep_deep, "w+", encoding='utf-8') as fout:
    for line in f:
        line = preprocessing_deep(line)
        fout.write(line)

with open(neg_file_path, encoding='utf8') as f, open(neg_file_path_prep_deep, "w+", encoding='utf-8') as fout:
    for line in f:
        line = preprocessing_deep(line)
        fout.write(line)

with open(test_file_path, encoding='utf8') as f, open(test_file_path_prep_deep, "w+", encoding='utf-8') as fout:
    for line in f:        
        line = line.partition(',')[2]             
        line = preprocessing_deep(line)
        if line == "": 
          fout.write("nothing \n")     
        else:   
           fout.write(line)

In [70]:
# print preprocessed dataset for deep learning models and debug
with open(test_file_path_prep_deep, encoding='utf8') as f:
    content = f.readlines()
data = [x.strip() for x in content]
data = pd.DataFrame(data)
data

,0
0,sea doo pro sea scooter sports with the portab...
1,shucks well i work all week so now i cannot co...
2,i cant stay away from bug thats my baby
3,no maam ! ! ! good im perfectly fine and not c...
4,whenever i fall asleep watching the tv i alway...
...,...
9995,had a nice time w my friend lastnite
9996,no it is not ! please stop !
9997,not without my daughter dvd twotime oscar r wi...
9998,have fun in class sweetcheeks


In [71]:
# preprocessing for other learning models: with word stemming

def preprocessing(line):
    line = replace_emoticons(line)
    line = rewrite_abbreviations(line)
    line = remove_hashtags(line)
    line = replace_numbers(line)        
    line = remove_stop_word(line)
    line = remove_punctuations(line)
    line = remove_spaces(line)
    line = stemming_words(line)
    return line

In [72]:
with open(pos_file_path, encoding='utf8') as f, open(pos_file_path_prep, "w+", encoding='utf-8') as fout:
    for line in f:
        line = preprocessing(line)
        if line == "":
          fout.write(line)
        else:
          fout.write(line + '\n')

with open(neg_file_path, encoding='utf8') as f, open(neg_file_path_prep, "w+", encoding='utf-8') as fout:
    for line in f:
        line = preprocessing(line)
        if line == "":
          fout.write(line)
        else:
          fout.write(line + '\n')

with open(test_file_path, encoding='utf8') as f, open(test_file_path_prep, "w+", encoding='utf-8') as fout:
    for line in f:        
        line = line.partition(',')[2]             
        line = preprocessing(line)
        if line == "": 
          fout.write("nothing \n")     
        else:   
           fout.write(line + '\n')

In [73]:
# print preprocessed dataset models and debug
with open(test_file_path_prep, encoding='utf8') as f:
    content = f.readlines()
data = [x.strip() for x in content]
data = pd.DataFrame(data)
data

,0
0,sea doo pro sea scooter sport with the portabl...
1,shuck well i work all week so now i cannot com...
2,i cant stay away from bug that my babi
3,no maam ! ! ! good im perfectli fine and not c...
4,whenev i fall asleep watch the tv i alway wake...
...,...
9995,had a nice time w my friend lastnit
9996,no it is not ! pleas stop !
9997,not without my daughter dvd twotim oscar r win...
9998,have fun in class sweetcheek
